In [1]:
import io
import json
import purly
import asyncio
import contextlib
import traceback
from multiprocessing import Process
from sanic import Sanic, response
from sanic_cors import cross_origin

In [2]:
@contextlib.contextmanager
def output_to_layout(layout):
    with io.StringIO() as buffer, contextlib.redirect_stdout(buffer):
        try:
            yield
        except:
            errors = traceback.format_exc()
        else:
            errors = ""
        finally:
            raw_output = buffer.getvalue() + errors
            if raw_output:
                output = layout.html("pre", layout.html("code", raw_output))
                layout.children.append(output)
                layout.sync()

In [3]:
purly.state.Machine(cors=True).daemon()

[2018-10-03 07:32:17 -0700] [11250] [INFO] Goin' Fast @ http://127.0.0.1:8000
[2018-10-03 07:32:17 -0700] [11250] [INFO] Starting worker [11250]


In [4]:
sandbox = Sanic()

begin = asyncio.Event()
begin.set()
release = asyncio.Event()


@sandbox.route("/exec", methods=["POST"])
@cross_origin(sandbox)
async def to_exe(request):
    layout = purly.Layout("ws://127.0.0.1:8000/model/sandbox/stream")
    output = purly.Layout("ws://127.0.0.1:8000/model/sandbox-output/stream")

    global release
    global begin

    release.set()
    await begin.wait()
    
    begin = asyncio.Event()
    release = asyncio.Event()
    
    loop = asyncio.get_event_loop()

    def _serve():
        if not release.is_set():
            loop.call_soon(_serve)
            with output_to_layout(output):
                layout.sync()
        else:
            layout.children.clear()
            output.children.clear()
            layout.sync()
            output.sync()
            begin.set()

    with output_to_layout(output):
        exec(request.json["code"], {"layout": layout})

    loop.call_soon(_serve)

    return response.json({})

In [5]:
Process(target=sandbox.run, kwargs=dict(port=8001), daemon=True).start()

[2018-10-03 07:32:19 -0700] [11259] [INFO] Goin' Fast @ http://127.0.0.1:8001
[2018-10-03 07:32:19 -0700] [11259] [INFO] Starting worker [11259]
[2018-10-03 07:32:39 -0700] - (sanic.access)[INFO][127.0.0.1:56414]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:32:54 -0700] - (sanic.access)[INFO][127.0.0.1:56422]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:33:08 -0700] - (sanic.access)[INFO][127.0.0.1:56432]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:33:13 -0700] - (sanic.access)[INFO][127.0.0.1:56432]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:33:22 -0700] - (sanic.access)[INFO][127.0.0.1:56446]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:34:16 -0700] - (sanic.access)[INFO][127.0.0.1:56470]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:34:21 -0700] - (sanic.access)[INFO][127.0.0.1:56478]: POST http://127.0.0.1:8001/exec  200 2
[2018-10-03 07:35:38 -0700] - (sanic.access)[INFO][127.0.0.1:56486]: POST http://127.0.0.1:8001/exe

In [13]:
div = layout.html('div')
layout.children.append(div)
div.style.update(height='20px', width='20px', backgroundColor='red')

@div.on('Click')
def toggle():
    print("changed")
    if div.style['backgroundColor'] == 'blue':
        div.style['backgroundColor'] = 'red'
    else:
        div.style['backgroundColor'] = 'blue'

In [ ]:
def hsl(radius, x, y):
    """Return an HSL color string."""
    x -= radius
    y -= radius
    unit_radius = int((x ** 2 + y **2) ** 0.5) / radius
    degrees = int(math.atan2(x, y) * 180 / math.pi)
    return "hsl(%s, 100%%, %s%%)" % (degrees, unit_radius * 100)


radius = 50
wheel = layout.html('div')
wheel.style.update(
    height="%spx" % (radius * 2),
    width="%spx" % (radius * 2),
    backgroundColor="hsl(120, 100%, 50%)",
    borderRadius="50%",
)


selection = layout.html('div')
selection.style.update(
    height="20px",
    width="20px",
    backgroundColor="hsl(120, 100%, 50%)",
)


layout.children.append(wheel)
layout.children.append(selection)
layout.sync()
layout


@wheel.on('MouseMove')
def cause_color_change(offsetX, offsetY):
    wheel.style["backgroundColor"] = hsl(50, offsetX, offsetY)

@wheel.on("Click")
def cause_color_select(offsetX, offsetY):
    selection.style["backgroundColor"] = hsl(50, offsetX, offsetY)